### Create the environment
To do so, we select an API endpoint and collect a token from our configuration. <br>
We also need an access token, in this example we use [oidc-agent](https://github.com/indigo-dc/oidc-agent#:~:text=oidc%2Dagent%20is%20a%20set,session%20or%20a%20login%20session.) to get one. <br>

In [1]:
eosc_perf_api="https://perf-stage.test.fedcloud.eu/api/v1"
access_token=$(oidc-token egi-demo)

### (Conditional) Register, if not done already
To use our service as user, first we need to accept the terms of usage and register. <br>
Make sure to read the [terms and conditions](https://performance.services.fedcloud.eu/privacy_policy).

In [2]:
curl -X 'POST' \
  "$eosc_perf_api/users:register" \
  -H "Authorization: Bearer $access_token"

### Search for the benchmark id that produced our result
You can get a list of all available benchmarks using GET /benchmarks
> Benchmarks are public to the Internet, access token is not needed

In [3]:
benchmarks=$(curl -X 'GET' "$eosc_perf_api/benchmarks?docker_image=deephdc/deep-oc-benchmarks_cnn&docker_tag=gpu")
benchmark=$(echo $benchmarks | jq '.items[0]')
echo $benchmark | jq '.json_schema = "..."'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   943  100   943    0     0   8980      0 --:--:-- --:--:-- --:--:--  8980
{
  "description": "A free description for the community",
  "docker_image": "deephdc/deep-oc-benchmarks_cnn",
  "docker_tag": "gpu",
  "id": "84b21e6d-9efe-44ed-b59c-c487a626cc33",
  "json_schema": "...",
  "upload_datetime": "2022-09-16T09:48:37.361474"
}


### Search for the flavor id used to run our benchmark
First you need to find the site where the benchmark was run. <br>
Once the site id is collected, it is possible to access and select the site flavors. <br>
> In this example we will use ':search' endpoint to find the flavor

In [4]:
sites=$(curl -X 'GET' "$eosc_perf_api/sites?name=ForHLR2")
site=$(echo $sites | jq '.items[0]')
echo $site | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   378  100   378    0     0   3532      0 --:--:-- --:--:-- --:--:--  3532
{
  "address": "https://wiki.scc.kit.edu/hpc/index.php?title=Category:ForHLR",
  "description": "Baden-Württemberg's general purpose tier 2 high performance computing (HPC) cluster",
  "id": "27ee87ff-a9aa-4486-891a-aff8164a9cab",
  "name": "ForHLR2",
  "upload_datetime": "2021-10-13T15:11:17.437972"
}


In [5]:
site_id=$(echo $site | jq -r '.id')
flavors=$(curl -X 'GET' "$eosc_perf_api/sites/$site_id/flavors:search?terms=haicore-gpu8")
flavor=$(echo $flavors | jq '.items[0]')
echo $flavor | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   239  100   239    0     0   2212      0 --:--:-- --:--:-- --:--:--  2212
{
  "description": "8x NVIDIA A100-40",
  "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
  "name": "haicore-gpu8",
  "upload_datetime": "2021-10-13T15:35:04.713245"
}


### Search for the tags to relate your result
Collect the tags you want to link to your result so users can find it easily. <br>
> If you do not know the name of the tag you can use :search as generic filter.

In [6]:
tag_gpu=$(curl -X 'GET' "$eosc_perf_api/tags?name=gpu" | jq '.items[0]')
echo $tag_gpu | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   166  100   166    0     0   1693      0 --:--:-- --:--:-- --:--:--  1693
{
  "description": "",
  "id": "f52d83ba-daa7-4d5c-899e-ae5d5c2487ad",
  "name": "gpu"
}


### Upload your result
Use an execution datetime and the collected site_id and flavor_id to upload a result. <br>

In [7]:
execution_datetime="2021-09-08T20:37:10.192459Z"
benchmark_id=$(echo $benchmark | jq -r '.id')
flavor_id=$(echo $flavor | jq -r '.id')
tag_gpu_id=$(echo $tag_gpu | jq -r '.id')
result_json='{"start_datetime": "2021-09-08T20:37:10.192459Z", "end_datetime": "2021-09-08T20:39:21.652488Z", "machine": {"cpus": 8, "ram": 32000}}'

In [8]:
query="execution_datetime=$execution_datetime&benchmark_id=$benchmark_id&flavor_id=$flavor_id&tags_ids=$tag_gpu_id"
curl -X 'POST' "$eosc_perf_api/results?$query" \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $access_token" \
  -H 'Content-Type: application/json' \
  -d "$result_json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1814  100  1680  100   134  11200    893 --:--:-- --:--:-- --:--:-- 12093
{
  "benchmark": {
    "description": "A free description for the community",
    "docker_image": "deephdc/deep-oc-benchmarks_cnn",
    "docker_tag": "gpu",
    "id": "84b21e6d-9efe-44ed-b59c-c487a626cc33",
    "json_schema": {
      "$id": "https://example.com/benchmark.schema.json",
      "$schema": "https://json-schema.org/draft/2019-09/schema",
      "properties": {
        "end_datetime": {
          "description": "The benchmark end datetime.",
          "format": "date-time",
          "type": "string"
        },
        "machine": {
          "description": "Execution machine details.",
          "properties": {
            "cpus": {
              "description": "Number of CPU.",
              "type": "integer"
            },
            "ram": 